In [1]:
!nvidia-smi

Thu Jul 15 16:07:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.06    Driver Version: 450.51.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0    38W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
moving_average_window = 7

In [4]:
total_df = pd.read_csv('./total.csv').drop('Unnamed: 0', axis=1)

In [5]:
for idx, country in enumerate(list(set(total_df['CountryCode']))):
    total_df.loc[total_df['CountryCode'] == country,'covid_confirmed'] = \
        total_df.loc[total_df['CountryCode'] == country,'covid_confirmed'].rolling(window=moving_average_window).mean()
    total_df.loc[total_df['CountryCode'] == country,'covid_deaths'] = \
        total_df.loc[total_df['CountryCode'] == country,'covid_deaths'].rolling(window=moving_average_window).mean()
    total_df.loc[total_df['CountryCode'] == country,'covid_recovered'] = \
        total_df.loc[total_df['CountryCode'] == country,'covid_recovered'].rolling(window=moving_average_window).mean()
    
    total_df.loc[total_df['CountryCode'] == country,'test_cumulTotal'] = \
        total_df.loc[total_df['CountryCode'] == country,'test_cumulTotal'].rolling(window=moving_average_window).mean()
    total_df.loc[total_df['CountryCode'] == country,'test_7dayMA'] = \
        total_df.loc[total_df['CountryCode'] == country,'test_7dayMA'].rolling(window=moving_average_window).mean()
    total_df.loc[total_df['CountryCode'] == country,'test_shortPosRate'] = \
        total_df.loc[total_df['CountryCode'] == country,'test_shortPosRate'].rolling(window=moving_average_window).mean()
    total_df.loc[total_df['CountryCode'] == country,'test_shortTestperCase'] = \
        total_df.loc[total_df['CountryCode'] == country,'test_shortTestperCase'].rolling(window=moving_average_window).mean()
    
    total_df.loc[total_df['CountryCode'] == country,'vac_first'] = \
        total_df.loc[total_df['CountryCode'] == country,'vac_first'].rolling(window=moving_average_window).mean()
    total_df.loc[total_df['CountryCode'] == country,'vac_second'] = \
        total_df.loc[total_df['CountryCode'] == country,'vac_second'].rolling(window=moving_average_window).mean()
    total_df.loc[total_df['CountryCode'] == country,'vac_daily'] = \
        total_df.loc[total_df['CountryCode'] == country,'vac_daily'].rolling(window=moving_average_window).mean()
    
    total_df.loc[total_df['CountryCode'] == country,'mob_retailRecreation'] = \
        total_df.loc[total_df['CountryCode'] == country,'mob_retailRecreation'].rolling(window=moving_average_window).mean()
    total_df.loc[total_df['CountryCode'] == country,'mob_groceryPharmacy'] = \
        total_df.loc[total_df['CountryCode'] == country,'mob_groceryPharmacy'].rolling(window=moving_average_window).mean()
    total_df.loc[total_df['CountryCode'] == country,'mob_parks'] = \
        total_df.loc[total_df['CountryCode'] == country,'mob_parks'].rolling(window=moving_average_window).mean()
    total_df.loc[total_df['CountryCode'] == country,'mob_transit'] = \
        total_df.loc[total_df['CountryCode'] == country,'mob_transit'].rolling(window=moving_average_window).mean()
    total_df.loc[total_df['CountryCode'] == country,'mob_work'] = \
        total_df.loc[total_df['CountryCode'] == country,'mob_work'].rolling(window=moving_average_window).mean()
    total_df.loc[total_df['CountryCode'] == country,'mob_resident'] = \
        total_df.loc[total_df['CountryCode'] == country,'mob_resident'].rolling(window=moving_average_window).mean()
    

In [6]:
total_df.dropna(inplace=True)
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52452 entries, 6 to 53027
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   CountryCode            52452 non-null  object 
 1   date                   52452 non-null  object 
 2   covid_confirmed        52452 non-null  float64
 3   covid_deaths           52452 non-null  float64
 4   covid_recovered        52452 non-null  float64
 5   test_cumulTotal        52452 non-null  float64
 6   test_7dayMA            52452 non-null  float64
 7   test_shortPosRate      52452 non-null  float64
 8   test_shortTestperCase  52452 non-null  float64
 9   vac_first              52452 non-null  float64
 10  vac_second             52452 non-null  float64
 11  vac_daily              52452 non-null  float64
 12  mob_retailRecreation   52452 non-null  float64
 13  mob_groceryPharmacy    52452 non-null  float64
 14  mob_parks              52452 non-null  float64
 15  mo

In [7]:
# prepare dataset
window = 56
predict = 28

data_x = list()
data_y = list()



for idx, country in enumerate(list(set(total_df['CountryCode']))):
    cur_df = total_df[total_df['CountryCode'] == country]
    print('.', end='')
    
    time_range = sorted(set(cur_df['date']))
    for time_idx in range(len(time_range)-(window+predict) + 1):
        view_start_date = time_range[time_idx]
        view_end_date = time_range[time_idx+(window-1)]
        
        pred_start_date = time_range[time_idx+window]
        pred_end_date = time_range[time_idx+(window+predict-1)]
        
        data = cur_df[(view_start_date <= cur_df['date']) & (cur_df['date'] <= pred_end_date)][[
            'covid_confirmed',
            'covid_deaths',
            'covid_recovered',
            'test_cumulTotal',
            'test_7dayMA',
            'test_shortPosRate',
            'test_shortTestperCase',
            'vac_first',
            'vac_second',
            'vac_daily',
            'mob_retailRecreation',
            'mob_groceryPharmacy',
            'mob_parks',
            'mob_transit',
            'mob_work',
            'mob_resident',
        ]]
        
        normalised_data = list()
        for feature in [
            'covid_confirmed',
            'covid_deaths',
            'covid_recovered',
            'test_cumulTotal',
            'test_7dayMA',
            'test_shortPosRate',
            'test_shortTestperCase',
            'vac_first',
            'vac_second',
            'vac_daily',
            'mob_retailRecreation',
            'mob_groceryPharmacy',
            'mob_parks',
            'mob_transit',
            'mob_work',
            'mob_resident',
        ]:
            scaler = preprocessing.StandardScaler().fit(data[feature].to_numpy().reshape(-1, 1))
            scaled_data = scaler.transform(data[feature].to_numpy().reshape(-1, 1))
            normalised_data.append(scaled_data.reshape(1, -1)[0])

        normalised_data = np.transpose(np.array(normalised_data))

        x = np.array(normalised_data[:window, :])
        y = np.array(normalised_data[window:window+predict, 0])

        data_x.append(x)
        data_y.append(y)    


................................................................................................

In [8]:
# (data size, time_steps, features)

np_x = np.array(data_x)
np_y = np.array(data_y)
print(np_x.shape)
print(np_y.shape)

(43270, 56, 16)
(43270, 28)


In [9]:
x_train, x_test, y_train, y_test = train_test_split(np_x, np_y, test_size=0.2, shuffle=True, random_state=42)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.25, shuffle=True, random_state=42)

In [10]:
print(x_train.shape, x_valid.shape, x_test.shape)
print(y_train.shape, y_valid.shape, y_test.shape)

(25962, 56, 16) (8654, 56, 16) (8654, 56, 16)
(25962, 28) (8654, 28) (8654, 28)


# Modelling

In [17]:
batch_size = 512

In [18]:
model = keras.Sequential()
model.add(layers.Conv1D(filters=32, kernel_size=7, padding='SAME', input_shape=(window, 16)))
model.add(layers.Conv1D(filters=64, kernel_size=14, padding='SAME'))
model.add(layers.Conv1D(filters=64, kernel_size=7, padding='SAME'))
model.add(layers.Conv1D(filters=64, kernel_size=14, padding='SAME'))
model.add(layers.Conv1D(filters=128, kernel_size=7, padding='SAME'))
model.add(layers.Conv1D(filters=128, kernel_size=14, padding='SAME'))

model.add(layers.LSTM(units=64, activation='tanh', return_sequences=False))

model.add(layers.Dropout(rate=0.2))

model.add(layers.Dense(64))
model.add(layers.Dense(32))
model.add(layers.Dense(predict))

model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 56, 32)            3616      
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 56, 64)            28736     
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 56, 64)            28736     
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 56, 64)            57408     
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 56, 128)           57472     
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 56, 128)           229504    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)               

In [19]:
# early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# 모델 학습과정 설정 
model.compile(loss='mse', optimizer='adam', metrics=['acc', 'mae', 'mape', 'mse'])

# 모델 트레이닝 
history = model.fit(x_train, y_train, validation_data=(x_valid, y_valid), 
                     epochs=1000, batch_size=batch_size, callbacks=[early_stopping], verbose=1)

Epoch 1/1000
51/51 [==============================] - ETA: 0s - loss: 0.8421 - acc: 0.0979 - mae: 0.7112 - mape: 92447.8184 - mse: 0.8421

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

# summarize history for mae
plt.plot(history.history['mae'])
plt.plot(history.history['val_mae'])
plt.title('model mae')
plt.ylabel('mae')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

# summarize history for mape
plt.plot(history.history['mape'])
plt.plot(history.history['val_mape'])
plt.title('model mape')
plt.ylabel('mape')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
from random import randint
idx = randint(0, len(x_test))
pred = model.predict(x_test[idx].reshape(1, window, 16))

import matplotlib.pyplot as plt

plt.figure(figsize=(12, 9))
plt.plot(y_test[idx], label = 'actual')
plt.plot(pred.reshape(-1, 1), label = 'prediction')
plt.legend()
plt.show()